In [ ]:
import pandas as pd

# File paths
input_file = r'C:\Users\Nelson Tausk\Dataset_dataproject\pol.csv'
output_folder = r'C:\Users\Nelson Tausk\Dataset_dataproject\render_4d'

chunk_size = 10000000 # Number of rows per chunk
columns_to_extract = [4, 22]  # Indices of columns 4 and 23

# Regex pattern to match strings containing ">>"
pattern = r">>"

# Process the file in chunks
chunks = pd.read_csv(
    input_file,
    engine='python',
    chunksize=chunk_size,
    on_bad_lines='skip',
    delimiter=',',
    quoting=3
)

for i, chunk in enumerate(chunks):
    # Extract columns 4 and 23 (indices 3 and 22) and drop missing values
    selected_columns = chunk.iloc[:, columns_to_extract].copy()
    selected_columns.columns = ['time', 'comment']  # Rename the columns for clarity
    selected_columns = selected_columns.dropna()

    # Filter out rows where 'comment' contains ">>"
    cleaned_data = selected_columns[~selected_columns['comment'].str.contains(pattern, na=False)]

    # Define the output file path for this chunk
    output_file = f"{output_folder}/chunk_{i + 1}.csv"

    # Save the cleaned chunk to a CSV file
    cleaned_data.to_csv(output_file, index=False, header=True)
    print(f"Cleaned chunk {i + 1} saved to {output_file}.")

print("All chunks processed and saved successfully.")